In [1]:
!pip3 install pyyaml


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip3 install python-csv

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------------------- -------------------- 0.8/1.6 MB 3.0 MB/s eta 0:00:01
   -------------------------------- ------- 1.3/1.6 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import yaml
import csv
import glob
import os
import pandas as pd
#import app_common.db_operations as db_ops

data_file_path =  r'F:\guvi\Capstone Projects\STOCK-project\DATA'
yaml_file_path = data_file_path + r'\yaml'
csv_file_path = data_file_path + r'\csv'

list_df = []
folders = os.listdir(yaml_file_path)

for folder in folders:
    path = yaml_file_path+"\\"+folder+"\\*.yaml"
    files_list = glob.glob(path)

    for i, each_yaml_file in enumerate(files_list):
        n = len(files_list)
        #print("Processing file {} of {} file name: {}".format(i+1, n ,each_yaml_file))

        with open(each_yaml_file) as file:
            data = yaml.safe_load(file)
            # data is in list of dict format
            # type(data) => <class 'list'>
            # type(data[0]) => <class 'dict'>
            df = pd.DataFrame(data)
            list_df.append(df)

#concatenate the monthly data
merged_df = pd.concat(list_df)
filepath = f"{csv_file_path}\\stocks.csv"
merged_df.to_csv(filepath,index=False)
print(f"Stocks.csv: file created: {filepath} ")
# grouping the data based on ticker
grouped_df = merged_df.groupby('Ticker')
# creating the csv for each ticker
for ticker, ticker_df in grouped_df:
    filepath = f"{csv_file_path}\\{ticker}.csv"
    ticker_df.to_csv(filepath,index=False)
    print(f"Ticker: {ticker} file created: {filepath} ")

# storing the data in the MySQL
#db_ops.export_to_sql(merged_df)
export_to_sql(merged_df)

Stocks.csv: file created: F:\guvi\Capstone Projects\STOCK-project\DATA\csv\stocks.csv 
Ticker: ADANIENT file created: F:\guvi\Capstone Projects\STOCK-project\DATA\csv\ADANIENT.csv 
Ticker: ADANIPORTS file created: F:\guvi\Capstone Projects\STOCK-project\DATA\csv\ADANIPORTS.csv 
Ticker: APOLLOHOSP file created: F:\guvi\Capstone Projects\STOCK-project\DATA\csv\APOLLOHOSP.csv 
Ticker: ASIANPAINT file created: F:\guvi\Capstone Projects\STOCK-project\DATA\csv\ASIANPAINT.csv 
Ticker: AXISBANK file created: F:\guvi\Capstone Projects\STOCK-project\DATA\csv\AXISBANK.csv 
Ticker: BAJAJ-AUTO file created: F:\guvi\Capstone Projects\STOCK-project\DATA\csv\BAJAJ-AUTO.csv 
Ticker: BAJAJFINSV file created: F:\guvi\Capstone Projects\STOCK-project\DATA\csv\BAJAJFINSV.csv 
Ticker: BAJFINANCE file created: F:\guvi\Capstone Projects\STOCK-project\DATA\csv\BAJFINANCE.csv 
Ticker: BEL file created: F:\guvi\Capstone Projects\STOCK-project\DATA\csv\BEL.csv 
Ticker: BHARTIARTL file created: F:\guvi\Capstone Pro

In [17]:

export_to_sql(merged_df)

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd

def export_to_sql(df):
    db_user = 'root'
    db_password = '12345678'
    db_host = 'localhost' 
    db_name = 'stocks'

    engine_url = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}"

    try:
        engine = sqlalchemy.create_engine(engine_url)
        df.to_sql(name='stocks_data', con=engine, if_exists='replace', index=False)
        print("Data have been imported into database")
    except Exception as e:
        print(f"Exception has thrown as below: \n {e} ")


In [18]:
import mysql.connector

try:
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="12345678"
    )

    cursor = connection.cursor()

    cursor.execute("CREATE DATABASE IF NOT EXISTS stocks;")

    # create table
    create_stocks_table = '''create table IF NOT EXISTS stocks_data 
    ( 
    ticker nvarchar(100), day DATETIME, month nvarchar(10),
    open float, close float, high float,low float, volume float

    ) '''

    cursor.execute("USE stocks")
    cursor.execute(create_stocks_table)

    connection.commit()
    print('Database  Tables created successfully')
except Exception as e:
    print(f"Exception thrown \n {e}")

Database  Tables created successfully
